# Experiment 04

This Experiment is the main experiment of the work, consists in run an optimized Random Forest in order to try to predict the values in question.(https://www.researchgate.net/publication/221533996_From_an_artificial_neural_network_to_a_stock_market_day-trading_system_A_case_study_on_the_BMF_BOVESPA).

This method consists in using the sliding window 

In [1]:
import numpy as np
import pandas as pd

import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor

### Constants and Parameters

In [2]:
dfColumns = ['date','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']
col15 = ['open','d-5_high','d-5_low','d-4_high','d-4_low','d-3_high','d-3_low','d-2_high','d-2_low','d-1_high','d-1_low','EMA_high','EMA_low','UB_close','LB_close']
col25 = ['open','d-5_open','d-5_high','d-5_low','d-5_close','d-4_open','d-4_high','d-4_low','d-4_close','d-3_open','d-3_high','d-3_low','d-3_close','d-2_open','d-2_high','d-2_low','d-2_close','d-1_open','d-1_high','d-1_low','d-1_close','EMA_high' ,'EMA_low','UB_close','LB_close']
DAYS = 5

In [3]:
TEST_BEGIN = '2020.06.01'
TEST_END = '2021.06.01'

WINDOW = 400
DRIFT = 10

TIMES = 1

def safeDate(df, begin, period):
    index = df[df['date'] == begin].index[0]
    
    newIndex = index - period*2
    newDate = df[df.index == newIndex].date
    
    return newDate.iloc[0]

### Opening the dataset

In [4]:
df = pd.read_csv('../Data/PETR4_Daily.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

beginDate = safeDate(df, TEST_BEGIN, WINDOW)

df2 = pd.read_csv('../Data/VALE3_Daily.csv', sep = "\t")
df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

In [5]:
#PETR4
df = calculateEMA(df,DAYS,newColumns)
df = calculateBB(df,DAYS,newColumns)
df = addPreviousDays(df,DAYS, newColumns)

#VALE3
df2 = calculateEMA(df2,DAYS,newColumns)
df2 = calculateBB(df2,DAYS,newColumns)
df2 = addPreviousDays(df2,DAYS, newColumns)

In [6]:
initialDf_PETR4 = getPeriod(df, beginDate ,TEST_END, True)
initialDf_VALE3 = getPeriod(df2,beginDate,TEST_END, True)

In [7]:
initialDf_PETR4

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2017.03.03,13.80,13.91,13.64,13.89,14.021268,14.099810,13.777194,13.937686,14.550400,...,13.70,13.77,14.46,14.60,14.04,14.11,14.51,14.53,14.12,14.24
1,2017.03.06,13.70,14.03,13.64,13.70,13.914179,14.076540,13.731463,13.858457,14.196798,...,13.89,14.08,13.92,14.03,13.70,13.77,14.46,14.60,14.04,14.11
2,2017.03.07,13.77,13.95,13.71,13.77,13.866119,14.034360,13.724309,13.828972,14.185301,...,13.60,13.70,14.07,14.10,13.89,14.08,13.92,14.03,13.70,13.77
3,2017.03.08,13.75,13.77,13.20,13.20,13.827413,13.946240,13.549539,13.619314,14.034911,...,13.64,13.89,14.00,14.01,13.60,13.70,14.07,14.10,13.89,14.08
4,2017.03.09,13.01,13.23,12.85,13.15,13.554942,13.707493,13.316359,13.462876,14.276313,...,13.64,13.70,13.80,13.91,13.64,13.89,14.00,14.01,13.60,13.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,2021.05.26,25.91,26.18,25.65,26.09,26.114791,26.354219,25.760657,26.042263,26.608719,...,25.94,25.95,26.15,26.28,25.72,25.93,25.85,26.35,25.80,26.15
1045,2021.05.27,26.08,26.25,25.77,25.91,26.103194,26.319479,25.763771,25.998175,26.604087,...,25.99,26.39,26.08,26.31,25.94,25.95,26.15,26.28,25.72,25.93
1046,2021.05.28,26.20,27.02,26.15,26.99,26.135462,26.552986,25.892514,26.328784,26.617551,...,25.78,25.84,26.28,26.52,25.99,26.39,26.08,26.31,25.94,25.95
1047,2021.05.31,26.96,27.20,26.78,26.87,26.410308,26.768657,26.188343,26.509189,27.147657,...,25.65,26.09,26.48,26.62,25.78,25.84,26.28,26.52,25.99,26.39


In [8]:
initialDf_VALE3

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2017.03.03,27.14,27.48,26.69,27.08,28.139791,28.542648,27.343451,27.673311,30.400962,...,27.23,27.84,29.95,30.00,27.91,28.23,30.07,30.07,28.96,29.52
1,2017.03.06,26.72,26.90,26.11,26.47,27.666527,27.995098,26.932301,27.272207,28.899205,...,27.96,28.23,27.36,28.86,27.23,27.84,29.95,30.00,27.91,28.23
2,2017.03.07,26.54,27.12,26.21,26.31,27.291018,27.703399,26.691534,26.951471,29.011901,...,26.68,26.77,28.13,28.41,27.96,28.23,27.36,28.86,27.23,27.84
3,2017.03.08,26.57,26.68,25.39,25.63,27.050679,27.362266,26.257689,26.510981,28.568944,...,26.69,27.08,28.34,28.59,26.68,26.77,28.13,28.41,27.96,28.23
4,2017.03.09,25.06,26.02,24.74,25.68,26.387119,26.914844,25.751793,26.233987,27.985215,...,26.11,26.47,27.14,27.48,26.69,27.08,28.34,28.59,26.68,26.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,2021.05.26,106.37,110.32,106.03,110.20,109.166674,111.131617,107.622606,109.807900,113.973861,...,108.50,109.40,111.70,112.10,110.20,111.11,112.10,112.78,109.84,112.25
1045,2021.05.27,110.80,112.09,109.77,110.99,109.711116,111.451078,108.338404,110.201933,113.405981,...,107.51,109.78,111.11,111.25,108.50,109.40,111.70,112.10,110.20,111.11
1046,2021.05.28,111.11,112.75,109.42,111.59,110.177411,111.884052,108.698936,110.664622,113.405981,...,106.78,107.05,108.00,110.14,107.51,109.78,111.11,111.25,108.50,109.40
1047,2021.05.31,112.50,114.89,112.50,114.78,110.951607,112.886035,109.965957,112.036415,114.811204,...,106.03,110.20,110.28,110.89,106.78,107.05,108.00,110.14,107.51,109.78


#### Experimento 04: Utilizando Random Forest

In [13]:
def getTrainPeriod(df, begin, period):
    
    index = df[df['date'] == begin].index[0]
    index = index-1
    
    endDate = df[df.index == index].date
    beginDate = df[df.index == (index-period)].date
    
    return beginDate.iloc[0], endDate.iloc[0]
    

def exp04(df, stock, inputColumns = []):
    
    beginTrain, endTrain = getTrainPeriod(df,TEST_BEGIN, WINDOW)
    
    df_test_aux = getPeriod(df, TEST_BEGIN, TEST_END)
    
    indexBegin = df_test_aux.head(1).index[0]
    indexEnd = df_test_aux.tail(1).index[0]
    
    for i in range (indexBegin, indexEnd, DRIFT):
        
        df_test = df[(df.index >= i) & (df.index <= i+DRIFT-1)]
        df_train = df[(df.index >= i-WINDOW) & (df.index < i)]
    
        if(inputColumns == []):
            x_train = df_train.drop(['date', 'high', 'low', 'close'], axis=1)
            y_train = df_train[['high','low']]

            x_test = df_test.drop(['date', 'high', 'low', 'close'], axis=1)
            y_test = df_test[['high','low']]
        else:
            x_train = df_train[inputColumns]
            y_train = df_train[['high','low']]

            x_test = df_test[inputColumns]
            y_test = df_test[['high','low']]

        rf = RandomForestRegressor(n_estimators = 500)

        rf.fit(x_train, y_train);
        y_pred = rf.predict(x_test)
        
        #print('train: ' + df_train['date'].head(1).tolist()[0] + ' - ' + df_train['date'].tail(1).tolist()[0])
        #print('_test: ' + df_test['date'].head(1).tolist()[0] + ' - ' + df_test['date'].tail(1).tolist()[0])
        #printResult(mape(y_test['high'],y_pred[:,0]), 'High')
        #printResult(mape(y_test['low'],y_pred[:,1]), 'Low')
        #print('###')
        
        if(i == indexBegin):
            max_test = np.array(y_test['high'])
            min_test = np.array(y_test['low'])
            max_pred = np.array(y_pred[:,0])
            min_pred = np.array(y_pred[:,1])
        else:
            max_test = np.append(max_test, np.array(y_test['high']))
            min_test = np.append(min_test, np.array(y_test['low']))
            max_pred = np.append(max_pred, np.array(y_pred[:,0]))
            min_pred = np.append(min_pred, np.array(y_pred[:,1]))
    
        #errorMax = mape(predDf['high'],predDf['high_pred'])
        #errorMin = mape(predDf['low'],predDf['low_pred'])
    
    return max_test, min_test, max_pred, min_pred

def test30timesExp04(df,stock, phrase, columnInput = []):
    errorMax = []
    errorMin = []
    accMax_test = []
    accMin_test = []
    accMax_pred = []
    accMin_pred = []
    
    for i in range(1, TIMES+1):
        if(columnInput == []):
            max_test, min_test, max_pred, min_pred = exp04(df, stock)
        else: 
            max_test, min_test, max_pred, min_pred = exp04(df, stock, columnInput)
    
        accMax_test = np.append(accMax_test,max_test)
        accMin_test = np.append(accMin_test,min_test)
        
        accMax_pred = np.append(accMax_pred,max_pred)
        accMin_pred = np.append(accMin_pred,min_pred)
    
    print(phrase)
    printResult(mape(accMax_test,accMax_pred), 'High')
    printResult(mape(accMin_test,accMin_pred), 'Low')

In [10]:
test30timesExp04(initialDf_PETR4, 'PETR4', 'PETR4 15 Variables', col15) 
test30timesExp04(initialDf_PETR4, 'PETR4', 'PETR4 25 Variables', col25) 
test30timesExp04(initialDf_PETR4, 'PETR4', 'PETR4 33 Variables') 
print('\n')
test30timesExp04(initialDf_VALE3, 'VALE3', 'VALE3 15 Variables', col15) 
test30timesExp04(initialDf_VALE3, 'VALE3', 'VALE3 25 Variables', col25) 
test30timesExp04(initialDf_VALE3, 'VALE3', 'VALE3 33 Variables')  

PETR4 15 Variables
High: 1.2 +- 0.98 %
Low: 1.23 +- 1.07 %
PETR4 25 Variables
High: 1.21 +- 1.0 %
Low: 1.25 +- 1.09 %
PETR4 33 Variables
High: 1.21 +- 1.0 %
Low: 1.27 +- 1.12 %


VALE3 15 Variables
High: 2.34 +- 2.81 %
Low: 2.41 +- 2.68 %
VALE3 25 Variables
High: 2.4 +- 2.82 %
Low: 2.47 +- 2.71 %
VALE3 33 Variables
High: 2.4 +- 2.79 %
Low: 2.48 +- 2.7 %


In [23]:
for x in range (100,701,100):
    WINDOW = x
    TIMES = 3
    print(WINDOW)
    test30timesExp04(initialDf_PETR4, 'PETR4', 'PETR4 15 Variables', col15) 
    test30timesExp04(initialDf_VALE3, 'VALE3', 'VALE3 15 Variables', col15) 
    print('#################')

100
PETR4 15 Variables
High: 2.24 +- 2.4 %
Low: 2.23 +- 2.74 %
VALE3 15 Variables
High: 2.38 +- 2.8 %
Low: 2.44 +- 2.69 %
#################
200
PETR4 15 Variables
High: 1.69 +- 1.7 %
Low: 1.79 +- 2.15 %
VALE3 15 Variables
High: 2.35 +- 2.81 %
Low: 2.39 +- 2.7 %
#################
300
PETR4 15 Variables
High: 1.36 +- 1.31 %
Low: 1.35 +- 1.48 %
VALE3 15 Variables
High: 2.36 +- 2.81 %
Low: 2.4 +- 2.7 %
#################
400
PETR4 15 Variables
High: 1.21 +- 0.98 %
Low: 1.23 +- 1.07 %
VALE3 15 Variables
High: 2.35 +- 2.82 %
Low: 2.42 +- 2.69 %
#################
500
PETR4 15 Variables
High: 1.09 +- 0.87 %
Low: 1.08 +- 0.98 %
VALE3 15 Variables
High: 2.24 +- 2.77 %
Low: 2.35 +- 2.66 %
#################
600
PETR4 15 Variables
High: 1.1 +- 0.88 %
Low: 1.02 +- 0.88 %
VALE3 15 Variables
High: 2.24 +- 2.77 %
Low: 2.34 +- 2.67 %
#################
700
PETR4 15 Variables
High: 1.04 +- 0.89 %
Low: 1.01 +- 0.9 %
VALE3 15 Variables
High: 2.25 +- 2.77 %
Low: 2.35 +- 2.67 %
#################


In [21]:
WINDOW = 700
TIMES = 3
print(WINDOW)
test30timesExp04(initialDf_PETR4, 'PETR4', 'PETR4 15 Variables',col15) 
test30timesExp04(initialDf_VALE3, 'VALE3', 'VALE3 15 Variables',col15) 
print('#################')

700
PETR4 15 Variables
High: 1.04 +- 0.89 %
Low: 1.02 +- 0.9 %
VALE3 15 Variables
High: 2.26 +- 2.78 %
Low: 2.36 +- 2.68 %
#################
